In [1]:
import numpy as np
train_st=np.load('./processed_data/station/train.npy')
train_lab=np.load('./processed_data/labels/train_labels.npy')
test_st=np.load('./processed_data/station/test.npy')
test_lab=np.load('./processed_data/labels/test_labels.npy')
no2_train_sat=np.load('./processed_data/no2_train.npy')
o3_test_sat=np.load('./processed_data/o3_test.npy')
no2_test_sat=np.load('./processed_data/no2_test.npy')
o3_train_sat=np.load('./processed_data/o3_train.npy')

In [2]:
train_st.shape

(17520, 16)

In [3]:
def log_standardize(x, mean=None, std=None):
    """
    Apply log1p transform, then standardize.
    
    Parameters
    ----------
    x : array-like
        Input values (e.g., ozone concentrations).
    mean : float, optional
        Mean from training set. If None, computed from x.
    std : float, optional
        Std from training set. If None, computed from x.
    
    Returns
    -------
    x_scaled : np.ndarray
        Log-transformed + standardized values.
    mean : float
        Mean used for scaling (store this for test data).
    std : float
        Std used for scaling (store this for test data).
    """
    x_log = np.log1p(x)
    
    if mean is None:
        mean = x_log.mean()
    if std is None:
        std = x_log.std()
        
    
    x_scaled = (x_log - mean) / std
    return x_scaled, mean, std

In [4]:
train_lab,train_mean,train_std=log_standardize(train_lab)
test_lab,_,_=log_standardize(test_lab,train_mean,train_std)

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

train_st=np.concatenate([train_st,train_lab],axis=1)
test_st=np.concatenate([test_st,test_lab],axis=1)
# Time-based split (assuming sequential data)
X_train=train_st[:-1]
y_train=train_lab[1:,1]
print(X_train.shape,y_train.shape)
X_test=test_st[:-1]
y_test=test_lab[1:,1]
# --- 2. Model Training ---
rf_model = GradientBoostingRegressor(
    n_estimators=100,      # Number of boosting stages
    learning_rate=0.1,     # Shrinks the contribution of each tree
    max_depth=3,           # Controls the size of the individual trees
    random_state=42
)
rf_model.fit(X_train, y_train)

# --- 3. Prediction and Evaluation ---
y_pred_rf = rf_model.predict(X_test)

# Calculate Evaluation Metrics
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Simple MAPE function
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_true)) * 100

mape_rf = calculate_mape(y_test, y_pred_rf)

# --- 4. Print Results ---
print(f"Random Forest Training Complete. Test set size: {len(X_test)}")
print("=" * 45)
print(f"R-squared (R²):                  {r2_rf:.4f}")
print(f"Root Mean Squared Error (RMSE):  {rmse_rf:.4f}")
print(f"Mean Absolute Error (MAE):       {mae_rf:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_rf:.2f}%")
print("=" * 45)

(17519, 20) (17519,)
Random Forest Training Complete. Test set size: 8783
R-squared (R²):                  0.7334
Root Mean Squared Error (RMSE):  0.4748
Mean Absolute Error (MAE):       0.3177
Mean Absolute Percentage Error (MAPE): 152.84%


In [6]:
y_pred_rf = rf_model.predict(X_train)

# Calculate Evaluation Metrics
rmse_rf = np.sqrt(mean_squared_error(y_train, y_pred_rf))
mae_rf = mean_absolute_error(y_train, y_pred_rf)
r2_rf = r2_score(y_train, y_pred_rf)

# Simple MAPE function
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_train - y_pred) / y_true)) * 100

mape_rf = calculate_mape(y_train, y_pred_rf)

# --- 4. Print Results ---
print(f"Random Forest Training Complete. Test set size: {len(X_train)}")
print("=" * 45)
print(f"R-squared (R²):                  {r2_rf:.4f}")
print(f"Root Mean Squared Error (RMSE):  {rmse_rf:.4f}")
print(f"Mean Absolute Error (MAE):       {mae_rf:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_rf:.2f}%")
print("=" * 45)

Random Forest Training Complete. Test set size: 17519
R-squared (R²):                  0.8735
Root Mean Squared Error (RMSE):  0.2546
Mean Absolute Error (MAE):       0.1704
Mean Absolute Percentage Error (MAPE): 107.29%
